# Evaluar retroactivamente recréditos

In [1]:
# importar librerias
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
datos = pd.read_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\recreditos_2024.csv", sep=";")
datos.sort_values(by="NO_CREDITO_NUEVO", inplace=True)
datos["SALDO"] = (datos["MONTO_DESEMBOLSADO_CANC"] - datos["MONTO_CANCELADO"]) / datos["MONTO_DESEMBOLSADO_CANC"]

In [3]:
cred_nuevos = datos[datos["SALDO"] >= 0.15]
prop_cred_nuevos = cred_nuevos["NO_CREDITO_NUEVO"].nunique() / datos["NO_CREDITO_NUEVO"].nunique()

# sacar proporcion de incumplimiento
recreditos = datos[datos["SALDO"] < 0.15]
prop_recred_obligatorio = recreditos["NO_CREDITO_NUEVO"].nunique() / datos["NO_CREDITO_NUEVO"].nunique()

In [16]:
print(f"La proporción de créditos que no cumple es de {prop_recred_obligatorio}")

La proporción de créditos que no cumple es de 0.6995592429349236


In [5]:
# función que determina si se pueden constituir reservas
def calcular_cuota(desembolso, plazo, tasa):
    try:
        cuota = (desembolso * tasa * (1 + tasa)**plazo) / ((1 + tasa)**plazo - 1)
    except:
        cuota = 0
    return cuota

def ingreso_por_intereses(n, desembolso, plazo, tasa):
    cuota = calcular_cuota(desembolso, plazo, tasa)
    try:
        ingresos = (desembolso - cuota / tasa)*((1 + tasa)**n - 1) + cuota*n
    except Exception as e:
        ingresos = 0
        print(e)
    return ingresos

def reservas_reestructuracion(desembolso):
    alpha = (0.08 + 0.0025) / (2 * (1 - 0.146))
    return (0.056 * 0.45 +  alpha) * desembolso

def constitucion_reservas(mes_inicio, mes_final, desembolso, plazo, tasa):
    alpha = (0.08 + 0.0025) / (2 * (1 - 0.146))
    ingresos_totales = ingreso_por_intereses(mes_final, desembolso, plazo, tasa) - ingreso_por_intereses(mes_inicio, desembolso, plazo, tasa)
    if (0.056 * 0.45 +  alpha) * desembolso <= ingresos_totales:
        return "Sí"
    else:
        return "No"


In [6]:
# aplicar flag para ver qué recréditos si pasaban
recreditos["FLAG RECREDITO"] = recreditos.apply(lambda x: constitucion_reservas(1, 6, x["MONTO_DESEMBOLSADO_NUEVO"], x["PLAZO_CRED_NUEVO"]/30, x["TASA_CRED_NUEVO"]/1200), axis=1)
recreditos["INGRESOS_MES6"] = recreditos.apply(lambda x: ingreso_por_intereses(6, x["MONTO_DESEMBOLSADO_NUEVO"], x["PLAZO_CRED_NUEVO"]/30, x["TASA_CRED_NUEVO"]/1200), axis=1)

# agrupar con no_credito_nuevo y solo pegar reservas en la primera ocurrencia
mascara_duplicados = recreditos.duplicated(subset="NO_CREDITO_NUEVO", keep="first")
alpha = (0.08 + 0.0025) / (2 * (1 - 0.146))
recreditos.loc[~mascara_duplicados, "RESERVAS_REESTRUCTURACION"] = (0.056*0.45 + alpha) * recreditos["MONTO_DESEMBOLSADO_NUEVO"]
recreditos.loc[mascara_duplicados, "RESERVAS_REESTRUCTURACION"] = 0
# Añadir verticalmetne los datos que faltaban
recreditos["CUOTA"] = recreditos.apply(lambda x: calcular_cuota(x["MONTO_DESEMBOLSADO_NUEVO"], x["PLAZO_CRED_NUEVO"]/30, x["TASA_CRED_NUEVO"]/1200), axis=1)

## Estudiar los que sí pasan la constitución de reservas

In [7]:
# ver el acumulado de reservas
recreditos.sort_values(by="NO_CREDITO_NUEVO", inplace=True)
recreditos["RESERVAS ACUMULADAS"] = recreditos["RESERVAS_REESTRUCTURACION"].cumsum()

## Evaluar reservas acumuladas

In [8]:
acumulado_reservas = 10000000
num = recreditos[recreditos["RESERVAS ACUMULADAS"] <= acumulado_reservas]["NO_CREDITO_NUEVO"].nunique()
den = recreditos["NO_CREDITO_NUEVO"].nunique()
prop = num / den
print(f"La proporción de número de recréditos que queda por debajo de la cota de constitución de reservas fue de {prop}")
prop_acumulado_universo = prop * prop_recred_obligatorio
print(f"La proporción de respecto del universo de recréditos es de {prop_acumulado_universo}")

La proporción de número de recréditos que queda por debajo de la cota de constitución de reservas fue de 0.06856422800385442
La proporción de respecto del universo de recréditos es de 0.04796473943479388


In [12]:
## ver proporcion de los que recuperan
prop_recred_reservas = recreditos[recreditos["FLAG RECREDITO"] == "Sí"]["NO_CREDITO_NUEVO"].nunique() / recreditos["NO_CREDITO_NUEVO"].nunique()
prop_reservas_universo = prop_recred_reservas * prop_recred_obligatorio

print(f"La proporción de recréditos que sí constituyen reservas es de {prop_recred_reservas}")
print(f"La proporción respecto del universo es de {prop_reservas_universo}")

La proporción de recréditos que sí constituyen reservas es de 0.6445037432362316
La proporción respecto del universo es de 0.45086855068706255


In [10]:
tpp_recreds = sum(recreditos["MONTO_DESEMBOLSADO_NUEVO"] * recreditos["TASA_CRED_NUEVO"]) / sum(recreditos["MONTO_DESEMBOLSADO_NUEVO"])
print(f"La TPP de recréditos es de {tpp_recreds}")

La TPP de recréditos es de 17.963966226242906


In [11]:
len(datos)

25504